In [55]:
import csv
from bs4 import BeautifulSoup
import pandas as pd
import requests
import datetime as dt

## Navigate To Website

In [56]:
# get to indeed.com
url = 'https://ca.indeed.com/jobs?q=data%20analyst&l&from=searchOnHP&vjk=0e5ecbfeb59b4abe'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'DNT': '1'
}

page = requests.get(url, headers=headers)

# create soup record
soup = BeautifulSoup(page.content, 'html.parser')
records = soup.find_all('td', class_='resultContent')

## Create Function to Extract Records

In [58]:
def get_record(record):

    title = record.h2.text
    company = record.find('span', class_='companyName').text
    location = record.find('div', class_='companyLocation').text

    # extract pay information if it exists
    try:
        pay = record.find('div', class_='metadata salary-snippet-container').text
    except AttributeError:
        pay = ''

    # extract rating informaiton if it extists
    try:
        rating = record.find('span', {'aria-hidden': 'true'}).text
    except AttributeError:
        rating = ''

    result = (title, company, location, pay, rating)

    return result


## Collect Records From All Pages

In [59]:
data = []
while True:
    try:
        url = 'https://ca.indeed.com' + soup.find('a', {'aria-label': 'Next'}).get('href')
    except AttributeError:
        break

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    records = soup.find_all('td', class_='resultContent')

    for card in records:
        record = get_record(card)
        data.append(record)


## Create Pandas Data.Frame

In [73]:
# covert to DataFrame
df = pd.DataFrame(data)

# set columns
df.columns = ["title", "company", "location", "pay", "rating"]

# file name
fname = 'data-analyst-' + dt.datetime.today().strftime('%Y-%m-%d') + '.csv'

# save to .csv
df.to_csv(fname)

# print first 5 rows of data
df.head()

,title,company,location,pay,rating
0,newJunior Data Analyst,AppleOne,"Mississauga, ON","$45,800 a year",3.8
1,Data Analyst (Remote in Canada),Interaxon,"Remote in Toronto, ON","$80,000 - $110,000 a year",
2,"newData Analyst (VBA,SQL)",Sun Life Financial,"Temporarily Remote in Toronto, ON",,3.9
3,Data Analyst/ Data Engineer/ Datawarehouse Ana...,Shorewise Consulting LLC,"Remote in Mississauga, ON",$47 an hour,
4,Data Analyst - Level 3,Libro Credit Union,"London, ON",,3.7
